In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
metadata_raw = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Video_Games", trust_remote_code=True)

In [3]:
metadata_raw_df = metadata_raw['full'].to_pandas()

In [4]:
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Video Games,Dash 8-300 Professional Add-On,5.0,1,[Features Dash 8-300 and 8-Q300 ('Q' rollout l...,[The Dash 8-300 Professional Add-On lets you p...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Aerosoft,"[Video Games, PC, Games]","{""Pricing"": ""The strikethrough price is the Li...",B000FH0MHO,None,None,None
1,Video Games,Phantasmagoria: A Puzzle of Flesh,4.1,18,[Windows 95],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sierra,"[Video Games, PC, Games]","{""Best Sellers Rank"": {""Video Games"": 137612, ...",B00069EVOG,None,None,None
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,4.3,223,[The #1 rated NBA video game simulation series...,[Following the record-breaking launch of NBA 2...,58.0,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['NBA 2K17 - Kobe: Haters vs Players...,2K,"[Video Games, PlayStation 4, Games]","{""Release date"": ""September 16, 2016"", ""Best S...",B00Z9TLVK0,None,None,None
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{""Best Sellers Rank"": {""Video Games"": 51019, ""...",B07SZJZV88,None,None,None
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,3.0,3,"[Includes (9) Total Accessories, Pedometer, Wi...",[The Thrustmaster Motion Plus Elite Fitness Pa...,None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","{""Release date"": ""November 1, 2009"", ""Pricing""...",B002WH4ZJG,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137264,None,Story of Seasons: Pioneers Of Olive Town (Nint...,4.5,397,[A wild world of discovery - tame the wilderne...,"[Product Description, Inspired by Tales of you...",31.04,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Marvelous Europe,"[Video Games, Nintendo Switch, Games]","{""Release date"": ""March 26, 2021"", ""Best Selle...",B09XQJS4CZ,None,None,None
137265,Video Games,MotoGP 18 (PC DVD) UK IMPORT REGION FREE,4.0,1,[Brand new game engine - MotoGP18 has been reb...,[Become the champion of the 2018 MotoGP Season...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Milestone,"[Video Games, Game Genre of the Month]","{""Pricing"": ""The strikethrough price is the Li...",B07DGPTGNV,None,None,None
137266,Cell Phones & Accessories,Century Accessory Soft Silicone Protective Ski...,2.9,19,"[Easy access to all buttons, controls and port...",[This soft case cover will add a splash of col...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Century Accessory,"[Video Games, Legacy Systems, Xbox Systems, Xb...","{""Package Dimensions"": ""2.76 x 2.76 x 0.2 inch...",B00HUWCQBW,None,None,None
137267,None,Hasbro Interactive Mr. Potato Head Activity Pa...,3.9,5,[],"[Amazon.com, Everyone's favorite master-of-dis...",None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Hasbro,"[Video Games, PC, Games]","{""Release date"": ""July 24, 1999"", ""Best Seller...",B00002S9MH,None,None,None


In [5]:
with pd.option_context('display.max_colwidth', None):
    display(metadata_raw_df.sample(10))

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
102065,Industrial & Scientific,Age of Empires III (Gold Edition),3.8,109,"[Age of Empires III (Gold Edition), Includes Age of Empires III, Also Includes Age of Empires III War Chiefs Expansion, All original manuals, CD's and box included, Perfect condition]",[],64.99,"{'hi_res': ['https://m.media-amazon.com/images/I/81Y0jXopZ6L._SL1000_.jpg'], 'large': ['https://m.media-amazon.com/images/I/61LqgMl79wL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/61LqgMl79wL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN']}","{'title': [], 'url': [], 'user_id': []}",Microsoft,"[Video Games, PC, Games]","{""Brand"": ""Microsoft"", ""Operating System"": ""See Description"", ""Hardware Platform"": ""Windows"", ""Format"": ""CD"", ""Genre"": ""Strategy Games""}",B000VWWKRI,None,None,None
74373,Video Games,The Elder Scrolls III: Morrowind (Certified Refurbished),3.8,101,[],[],15.99,"{'hi_res': ['https://m.media-amazon.com/images/I/81M9WsItFcL._SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/41oyWiqMGZL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/41oyWiqMGZL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN']}","{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, PlayStation Systems, PlayStation, Consoles]",{},B07L9FFT2F,None,None,None
37253,Video Games,Volume Flex Cable for PSP 1000,4.5,3,"[Brand new and high quality, A replacement flex ribbon for Start Home Volume Key Flex Cable, Solve the problem of these key defective, Easy to install and no need to pay for high repairing fee]",[Package included: 1 x Volume Flex Cable for PSP 1000],None,"{'hi_res': [None], 'large': ['https://m.media-amazon.com/images/I/31wmPXxqtwL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/31wmPXxqtwL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN']}","{'title': [], 'url': [], 'user_id': []}",Hi-eseller Store,"[Video Games, Legacy Systems, PlayStation Systems, Sony PSP, Accessories, Cables & Adapters, Cables]","{""Pricing"": ""The strikethrough price is the List Price. Savings represents a discount off the List Price."", ""Package Dimensions"": ""5.4 x 0.5 x 0.1 inches; 3.2 Ounces"", ""Is Discontinued By Manufacturer"": ""No"", ""Item Weight"": ""3.2 ounces"", ""Manufacturer"": ""Hi-eseller Store"", ""Date First Available"": ""November 3, 2010""}",B004AIV69C,None,None,None
42225,Computers,"ENHANCE Xbox Backpack - Xbox Carrying Case Compatible with Xbox One X, One S - Gear Arsenal Storage Compartments for Xbox Controllers, Gaming Headset, Xbox Games & More Xbox Accessories - Green",4.8,497,"[GAMING BACKPACK FOR CONSOLES & LAPTOPS: Features a dedicated storage compartment for Xbox , One X , laptops and VR headsets of all sizes with a padded divider for organization, ARSENAL STYLE ACCESSORY CACHE: Front opening storage with compartments of various sizes and styles to store controllers, cables & adapters, game discs, chargers and more, PADDED ARMOR PROTECTIVE DESIGN: The entire bag is outfitted with padded liners that offer resistance from impacts, bumps, scratches and crushing damage, MOISTURE WICKING & WEATHER RESISTANT: Keep your cool with the tessellated moisture resistant fabric on the backside of the backpack and a weather resistant base liner to protect from moisture, 3 YEAR WARRANTY: All ENHANCE Gaming products are backed by a 3 Year Manufacturer Warranty]",[],69.99,"{'hi_res': ['https://m.media-amazon.com/images/I/815-NRiM84L._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/81p8lvftivL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/91r8f7VWTZL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/91I58MSC7iL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/91k9f2E+jBL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/91rsBroRwpL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/81dgGXYzO3L._AC_SL1500_.jp

# Build Feature Transformations

In [6]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [7]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [8]:
# Custom function to reshape data from 1D to 2D
def reshape_2d_to_1d(X):
    return np.array(X).reshape(-1)

def flatten_string_array_col(X):
    return np.array(['\n'.join(x) for x in X])

def tfidf_pipeline_steps():
    steps = [
        ('impute', SimpleImputer(strategy='constant', fill_value='')),
        ('reshape', FunctionTransformer(reshape_2d_to_1d, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=500, ngram_range=(1, 2)))
    ]
    return steps

def description_pipeline_steps():
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=500, ngram_range=(1, 2)))
    ]
    return steps

def categories_pipeline_steps():
    def tokenizer(s):
        return s.split('\n')
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', CountVectorizer(tokenizer=tokenizer, token_pattern=None))
    ]
    return steps

data = pd.Series(["from 14.99", "14.99", "price: 9.99", "20 dollars", "none"])
price_pattern = r'\b((?:\d+\.\d*)|(?:\d+))\b'
display(data.str.extract(price_pattern))

def price_parse_dtype(series, pattern):
    return series.str.extract(pattern).astype(float)

def price_pipeline_steps(price_pattern):
    steps = [
        ('extract_price', FunctionTransformer(price_parse_dtype, kw_args=dict(pattern=price_pattern), validate=False)),
        ('impute', SimpleImputer(strategy='constant', fill_value=0)),
        ('min_max_scale', MinMaxScaler())
    ]
    return steps

,0
0,14.99
1,14.99
2,9.99
3,20
4,NaN


In [9]:
tfm = [
    ('main_category', OneHotEncoder(handle_unknown='ignore'), ['main_category']),  # One-hot encoding for categorical field
    ('title', Pipeline(tfidf_pipeline_steps()), ['title']),  # TF-IDF vectorizer for text field
    ('description', Pipeline(description_pipeline_steps()), 'description'),  # TF-IDF vectorizer for another text field
    ('categories', Pipeline(categories_pipeline_steps()), 'categories'),  # Count Vectorizer for multi-label categorical
    ('price', Pipeline(price_pipeline_steps(price_pattern)), 'price')  # Normalizing price
]
cols = [x[0] for x in tfm]
cols

['main_category', 'title', 'description', 'categories', 'price']

In [10]:
train_full_df = pd.merge(train_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
val_full_df = pd.merge(val_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
train_full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0050SVNZ8,4.0,1321885664000,Video Games,Amazon Basics Carrying Case for Nintendo - New...,[],"[Video Games, Legacy Systems, Nintendo Systems...",None
1,AHATA6X6MYTC3VNBFJ3WIYVK257A,B00LZVNWIA,4.0,1408233606000,Computers,Logitech G402 Hyperion Fury FPS Gaming Mouse,[Logitech G402 Hyperion Fury FPS Gaming Mouse],"[Video Games, PC, Accessories, Gaming Mice]",None
2,AHATA6X6MYTC3VNBFJ3WIYVK257A,B074RNL1RX,5.0,1511753174174,Video Games,Razer Wolverine Ultimate Officially Licensed X...,[Play anywhere with the Razer Wolverine Ultima...,"[Video Games, PC, Accessories, Controllers]",64.98
3,AHATA6X6MYTC3VNBFJ3WIYVK257A,B089QYP649,5.0,1531092820696,Video Games,Turtle Beach Stealth 600 Wireless Surround Sou...,[The Turtle Beach Stealth 600 is the latest wi...,"[Video Games, PlayStation 4, Accessories, Head...",168.75
4,AHATA6X6MYTC3VNBFJ3WIYVK257A,B07DHNX18W,4.0,1604348335046,Computers,Razer Huntsman Elite Gaming Keyboard: Fast Key...,[Introduces the new Razer Opto-Mechanical swit...,"[Video Games, PC, Accessories, Gaming Keyboards]",219.99
...,...,...,...,...,...,...,...,...,...
170040,AG6SXILM23AQQDRKFDI3D3IPGXPQ,B001T0HVGQ,4.0,1197406399000,Video Games,Xbox 360 Elite System Console Includes 120GB H...,"[Product Description, Xbox 360 sets a new pace...","[Video Games, Legacy Systems, Xbox Systems, Xb...",None
170041,AG6SXILM23AQQDRKFDI3D3IPGXPQ,B001EBBGTS,5.0,1204415647000,Video Games,Quake 4 - PC,"[Product Description, id Software’s QUAKE 4, d...","[Video Games, PC, Games]",32.99
170042,AG6SXILM23AQQDRKFDI3D3IPGXPQ,B000OCXK6A,4.0,1208216992000,Video Games,Frontlines: Fuel of War - Xbox 360,[Frontlines: Fuel of War is an open-world Firs...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",20.8
170043,AG6SXILM23AQQDRKFDI3D3IPGXPQ,B001G6062E,4.0,1214532340000,Video Games,Prey - Xbox 360,"[From the Manufacturer, Tommy is a Cherokee ga...","[Video Games, Legacy Systems, Xbox Systems, Xb...",31.9


In [11]:
train_full_df.to_parquet("../data/train_item_features.parquet")
val_full_df.to_parquet("../data/val_item_features.parquet")